In [ ]:
import scipy.io
import scipy.signal
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit, bisect
from scipy.stats import norm
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from nptdms import TdmsFile
import csv
from PythonOTtools import callTDMS, flip_molecule, TDMS_reader, gauss, bimodal, fwd_exponential, \
    rev_exponential, covariance_pds, bimodal_fit, bimodal_fit_singlethreshold, find_prelimEvents, \
    find_prelimEvents_singlethreshold, timeFilterEvents, plot_prelimEvents_withfilter, save_events_padded, save_events_ST_padded, \
    plot_prelimEvents_covthreshold_withfilter, find_event_npyfiles, extend_events_combined_covmethod

In [ ]:
#point to the name of the file. Our data is saved as .tdms files containing the bead positions and some metadata
file = 'Insert file name here'

#flip the molecule if necessary: 1 for flipped (negative-displacement) molecule, 0 for positive molecule
#to average multiple molecules, they must all point the same direction; by convention, all molecules should be positive
flipped_molecule = 0

#create a numpy array with all the information from the metadata etc
trace = callTDMS(file)

#flip the molecule
if flipped_molecule:
    trace = flip_molecule(trace)
    
#plot the bead positions to see how the trace looks
plt.figure(figsize=(10, 3))
plt.plot(trace[12], trace[7])
plt.plot(trace[12], trace[8])
plt.show()

In [ ]:
#calculate covariance over a window, 8 ms by default but can be set by changing cov_window
#calculating covariance can be computationally heavy, so this function saves the covariance
covar_AB = covariance_pds(trace, cov_window=8)
#generate a histogram of the covariances and plot it, so the peaks should be visibile
cov_hist = np.histogram(covar_AB, bins = 100)
plt.plot(cov_hist[1][:-1:], cov_hist[0])
plt.show()
plt.close('all')

In [ ]:
#do a bimodal fit of the covariance histogram, and save it in the same folder as the covariance
#this function is somewhat sensitive to initial guesses, which might need adjustment
#a1, m1, and s1 are the guesses for amplitude, mean, and standard deviation of the lower (bound) peak
#a2, m2, and s2 are the same but for the higher (unbound) peak
peaks = bimodal_fit(trace, covar_AB, a1 = 3000, m1=6, s1=0.2, a2 = 8000, m2=12, s2 = 5)

In [ ]:
#use the peaks from the bimodal fit to guess at preliminary events
prelimEvents = find_prelimEvents(covar_AB, peaks[0], peaks[1])
#filter out any events shorter than the dead time of the instrument (16ms) or closer than 16 ms from any other event
timeFilteredEvents = timeFilterEvents(trace, prelimEvents)
#plot the time filtered events, and save this trace
plot_prelimEvents_withfilter(trace, covar_AB, prelimEvents)

In [ ]:
#save the events as numpy arrays. Event start and ends are found by crossing a covariance threshold;
#this function pads the events by the amounts set in front_pad and back_pad (4000 points by default, or 16 ms)
save_events_padded(trace, timeFilteredEvents, covar_AB, front_pad = 4000, back_pad = 4000)

In [ ]:
#the above cells must be run for each individual trace prior to performing fuller analysis, which follows

In [ ]:
#find all the events related to a single molecule
M1 = 'folder containing a molecule data'
#this function finds all the files containing the strings 'covmethod_event' and '.npy'
#and returns a list that contains each event's file name
M1_events = find_event_npyfiles(M1, 'covmethod_event', '.npy')

In [ ]:
#join together multiple molecules
Mtot = M1_events + M2_events + M3_events #for any number of molecules, add more M_events

In [ ]:
#generate the event lifetimes
lifetime_list = lifetime_listgen(Mtot, front_pad=4000, back_pad=4000)

In [ ]:
#this function finds the start and stop point of events, as determined by covariance threshold
#this is saved as its own numpy array when the prelimEvents function is run
Mtot_eventtimes = find_event_times(Mtot)

In [ ]:
#This function calculates the time from the end of one event until the start of the next, returning a numpy array
Mtot_reattach = reattachmentrate(Mtot_eventtimes)

In [ ]:
#measure the step size
#this function saves numpy arrays and histograms of the individual steps
step_size_dist(Mtot)

In [ ]:
#TO GENERATE AN ENSEMBLE AVERAGE
#extend events to all be the same lifetime
#recall that the saved events are padded by 4000 pts (16 ms) by default
#this padding is considered in this function, still 4000 pts by default, but must be changed if initial padding is chaged
#postbind_exten and preUB_exten are 4 ms by default
#these set how many ms from the transition point to do the averaging for event extension
fwd_events_tot, rev_events_tot = extend_events_combined_covmethod(Mtot, front_pad = 4000, back_pad = 4000)
#this function returns two large numpy arrays, n x m, where n is the number of events and m is the padded length of the longest event

In [ ]:
#average all the extended events together and plot
#optionally, save it

#recall the front and back padding are 4000 pts by default, but can be changed
front_pad = 4000
back_pad = 4000
#frequency of our data collection in Hertz
freq = 250000
#the window of points that are included in the plot
exten_win = 4000
#set what the forward and reverse traces are, just the output of extend_events_combined_covmethod (large numpy arrays)
fwd_traces = fwd_events_tot
rev_traces = rev_events_tot
#create unpadded versions of the numpy arrays (ie, just the data between the start and stop of the event)
all_events_unpad_fwd = fwd_traces[...,front_pad:-back_pad]
all_events_unpad_rev = rev_traces[...,front_pad:-back_pad]

time_interval = 1/freq

#create a numpy array of time points, necessary for plotting
ext_event_time_padded = np.arange(0, len(fwd_traces[0])) * time_interval
#time array for the lengthened events, unpadded, necessary for fitting
ext_event_time = ext_event_time_padded[front_pad:-back_pad]
#generate guesses for the initial values for fitting exponentials
initial_params_fwd = [np.nanmean(all_events_unpad_fwd[:,-exten_win::])-np.nanmean(all_events_unpad_fwd[:,:exten_win:]),\
                60, np.nanmean(all_events_unpad_fwd[:,:exten_win:])]
initial_params_rev = [np.nanmean(all_events_unpad_rev[:,-exten_win::])-np.nanmean(all_events_unpad_rev[:,:exten_win:]),\
                8, np.nanmean(all_events_unpad_rev[:,:exten_win:])]
#fit forward (popt1) and reverse (popt2) exponentials
popt1, pcov1 = scipy.optimize.curve_fit(fwd_exponential, \
        xdata = ext_event_time, ydata = np.nanmean(all_events_unpad_fwd, axis = 0), p0 = initial_params_fwd, maxfev=8000)
popt2, pcov2 = scipy.optimize.curve_fit(rev_exponential, \
        xdata = ext_event_time, ydata = np.nanmean(all_events_unpad_rev, axis = 0), p0 = initial_params_rev)
#generate curves of fits
yfit_fwd = fwd_exponential(ext_event_time, *popt1)
yfit_rev = rev_exponential(ext_event_time, *popt2)
#plot the data and fits
plt.plot(ext_event_time_padded, np.nanmean(fwd_traces, axis = 0), color = 'dimgray', label = "Forward average")
plt.plot(ext_event_time_padded, np.nanmean(rev_traces, axis = 0), color = 'darkgray', label = "Reverse average")
plt.plot(ext_event_time, yfit_fwd, 'black', label = popt1[1])
plt.plot(ext_event_time, yfit_rev, 'black', label = popt2[1], alpha=0.6)
plt.legend(loc = 'lower right')
plt.gcf()
#save with the following function
#plt.savefig('/Users/bob/Desktop/20230411/EnsembleAverage.svg', format='svg')
plt.show()